<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [1]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_codigo.csv')



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [7]:
# FIXME

df = pd.DataFrame()

# a)
dfs = [pd.read_csv(url) for url in archivos_anio]

# Normalizar columnas
for i in range(len(dfs)):
    dfs[i].columns = dfs[i].columns.str.lower()

# Unir por filas
df_anio = pd.concat(dfs, ignore_index=True)

# Conversión de tipos numéricos
df_anio['anio']   = pd.to_numeric(df_anio['anio'], errors='coerce').astype('Int64')
df_anio['indice'] = pd.to_numeric(df_anio['indice'], errors='coerce')


#b)
# Revisar duplicados en codigo_iso
duplicados = df_codigos[df_codigos.duplicated(subset=["codigo_iso"], keep=False)]

print("Códigos ISO duplicados:")
print(duplicados)

#c)
df = pd.merge(df_anio, df_codigos, on='codigo_iso', how='inner')
df = df.rename(columns=str.lower)


Códigos ISO duplicados:
    codigo_iso      pais
179        ZWE  Zimbabue
180        ZWE      malo




### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [8]:
print("Filas:", df.shape[0])
print("Columnas:", df.shape[1])
print("Columnas:", df.columns.tolist())
print("\nTipos:\n", df.dtypes)
print("\nNulos por columna:\n", df.isnull().sum())
print("\nPaíses distintos:", df['pais'].nunique())
print("Años distintos:", df['anio'].nunique())

Filas: 3077
Columnas: 5
Columnas: ['codigo_iso', 'anio', 'indice', 'ranking', 'pais']

Tipos:
 codigo_iso     object
anio            Int64
indice        float64
ranking       float64
pais           object
dtype: object

Nulos por columna:
 codigo_iso      0
anio            0
indice        397
ranking       223
pais            0
dtype: int64

Países distintos: 180
Años distintos: 17





### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [9]:
# respuesta
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN']

df_america =  pd.DataFrame() # FIX ME

df_america = df[df['codigo_iso'].isin(america)].copy()
df_america['anio']   = pd.to_numeric(df_america['anio'], errors='coerce').astype('Int64')
df_america['indice'] = pd.to_numeric(df_america['indice'], errors='coerce')
df_america = df_america.dropna(subset=['anio','indice'])
df_america = df_america.groupby(['pais','anio'], as_index=False)['indice'].mean()
if df_america.empty:
    print("No hay datos válidos para América.")
else:
    for anio, sub in df_america.groupby('anio', sort=True):
        if sub.empty:
            continue
        idx_min = sub['indice'].idxmin()
        idx_max = sub['indice'].idxmax()
        pais_min = sub.loc[idx_min, 'pais']
        pais_max = sub.loc[idx_max, 'pais']
        val_min  = sub.loc[idx_min, 'indice']
        val_max  = sub.loc[idx_max, 'indice']
        print(f"Año {int(anio)}: menor índice -> {pais_min} ({val_min:.3f}), mayor índice -> {pais_max} ({val_max:.3f})")


if df_america.empty:
    print("No hay datos válidos para América.")
else:
    resultado = df_america.groupby('anio').apply(
        lambda g: pd.Series({
            'pais_menor_indice': g.loc[g['indice'].idxmin(), 'pais'],
            'pais_mayor_indice': g.loc[g['indice'].idxmax(), 'pais']
        })
    )
    print(resultado)



Año 2001: menor índice -> Canadá (0.800), mayor índice -> Cuba (90.300)
Año 2002: menor índice -> Trinidad y Tobago (1.000), mayor índice -> Cuba (97.830)
Año 2003: menor índice -> Trinidad y Tobago (2.000), mayor índice -> Argentina (35826.000)
Año 2004: menor índice -> Trinidad y Tobago (2.000), mayor índice -> Cuba (87.000)
Año 2005: menor índice -> Bolivia (4.500), mayor índice -> Cuba (95.000)
Año 2006: menor índice -> Canadá (4.880), mayor índice -> Cuba (96.170)
Año 2007: menor índice -> Canadá (3.330), mayor índice -> Cuba (88.330)
Año 2008: menor índice -> Canadá (3.700), mayor índice -> Cuba (94.000)
Año 2009: menor índice -> Estados Unidos (6.750), mayor índice -> Cuba (78.000)
Año 2012: menor índice -> Jamaica (9.880), mayor índice -> Cuba (71.640)
Año 2013: menor índice -> Jamaica (10.900), mayor índice -> Cuba (70.920)
Año 2014: menor índice -> Canadá (10.990), mayor índice -> Cuba (70.210)
Año 2015: menor índice -> Costa Rica (11.100), mayor índice -> Cuba (70.230)
Año 2

/tmp/ipython-input-3595769069.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  resultado = df_america.groupby('anio').apply(


### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [ ]:
# FIX ME


pivot = pd.pivot_table(
    df, index='pais', columns='anio', values='indice',
    aggfunc='max', fill_value=0
)

#a)
max_pais = pivot.max(axis=1).idxmax()
min_pais = pivot.replace(0, pd.NA).min(axis=1).idxmin()
print("Mayor índice:", max_pais, pivot.loc[max_pais].max())
print("Menor índice distinto de 0:", min_pais, pivot.loc[min_pais].replace(0, pd.NA).min())

#b)
promedios = pivot.mean(axis=0)
promedios = pivot.mean(axis=0)
print("Año con mayor promedio:", promedios.idxmax(), promedios.max())
print("Año con menor promedio:", promedios.idxmin(), promedios.min())

#c)
variabilidad = pivot.max(axis=1) - pivot.min(axis=1)
pais_var = variabilidad.idxmax()
print("Mayor variabilidad:", pais_var, variabilidad.max())

#d)
constantes = pivot.apply(lambda fila: fila.nunique() == 1, axis=1)
print("Constantes:", pivot.index[constantes].tolist())

#e)
todo_cero = pivot.eq(0).all(axis=1)
print("Países con todo 0:", pivot.index[todo_cero].tolist())